# Requirements

In [155]:
!pip install tenseal deepface

In [156]:
import tenseal as ts
from deepface import DeepFace
import base64

# Finding embeddings

Done in the client side.

In [157]:
img1_path = "enroll.jpg"
img2_path = "verify.jpg"

In [158]:
# Returns a list

img1_embedding = DeepFace.represent(img1_path, model_name = 'Facenet')
img2_embedding = DeepFace.represent(img2_path, model_name = 'Facenet')

In [159]:
enrolled_embedding = img1_embedding[0]['embedding']
verify_embedding = img2_embedding[0]['embedding']

# Commons

In [160]:
def write_data(file_name, data):

    if type(data) == bytes:
        #bytes to base64
        data = base64.b64encode(data)

    with open(file_name, 'wb') as f:
        f.write(data)

def read_data(file_name):
    with open(file_name, "rb") as f:
        data = f.read()

    #base64 to bytes
    return base64.b64decode(data)

# Initialization

Generate secret/public key pair. Done client side.

In [161]:
context = ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree = 8192,
            coeff_mod_bit_sizes = [60, 40, 40, 60]
          )

context.generate_galois_keys()
context.global_scale = 2**40

In [162]:
secret_context = context.serialize(save_secret_key = True)
write_data("secret.txt", secret_context)

In [163]:
context.make_context_public() #drop the secret_key from the context
public_context = context.serialize()
write_data("public.txt", public_context)

In [164]:
del context, secret_context, public_context

# Encryption

Apply homomorphic encryption to facial embeddings. Done client side.

Later, homomorphic encrypted facial embeddings will be stored in the cloud.

In [165]:
context = ts.context_from(read_data("secret.txt"))

In [166]:
enc_v1 = ts.ckks_vector(context, enrolled_embedding)
enc_v2 = ts.ckks_vector(context, verify_embedding)

In [167]:
enc_v1_proto = enc_v1.serialize()
enc_v2_proto = enc_v2.serialize()

In [168]:
write_data("enc_v1.txt", enc_v1_proto)
write_data("enc_v2.txt", enc_v2_proto)

In [169]:
del context, enc_v1, enc_v2, enc_v1_proto, enc_v2_proto

# Calculations

Once homomorphic encrypted facial embeddings stored in the cloud, we are able to make calculations on encrypted data.

Here, we only need the public key.

In [170]:
context = ts.context_from(read_data("public.txt"))

In [171]:
enc_v1_proto = read_data("enc_v1.txt")
enc_v2_proto = read_data("enc_v2.txt")

In [172]:
enc_v1 = ts.lazy_ckks_vector_from(enc_v1_proto)
enc_v1.link_context(context)

enc_v2 = ts.lazy_ckks_vector_from(enc_v2_proto)
enc_v2.link_context(context)

In [173]:
euclidean_squared = enc_v1 - enc_v2
euclidean_squared = euclidean_squared.dot(euclidean_squared)

In [174]:
write_data("euclidean_squared.txt", euclidean_squared.serialize())

In [175]:
# We must not decrypt the homomorphic encrypted euclidean squared value in this stage
#because we don't have the secret key.

try:
    euclidean_squared.decrypt()
except Exception as err:
    print("Exception: ", str(err))

Exception:  the current context of the tensor doesn't hold a secret_key, please provide one as argument


In [176]:
del context, enc_v1_proto, enc_v2_proto, enc_v1, enc_v2, euclidean_squared

# Decryption

Once homomorphic encrypted euclidean squared value found in the cloud, we are going to retrieve it to the client side.

Client can decrypt it because we have the secret key in the client side.

In [177]:
context = ts.context_from(read_data("secret.txt"))

In [178]:
euclidean_squared_proto = read_data("euclidean_squared.txt")

In [179]:
euclidean_squared = ts.lazy_ckks_vector_from(euclidean_squared_proto)
euclidean_squared.link_context(context)

In [180]:
euclidean_squared_plain = euclidean_squared.decrypt()[0]

In [181]:
euclidean_squared_plain

51.57489099802089

In [182]:
if euclidean_squared_plain < 100:
    print("they are same person")
else:
    print("they are different persons")

they are same person
